In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import GridSearchCV, train_test_split

np.random.seed(1)

In [2]:
train_df = pd.read_csv('./dataset/train.csv')
test_df = pd.read_csv('./dataset/test.csv')

In [ ]:
train_df['binary_week'] = pd.to_datetime(train_df['date_time']).dt.weekday
test_df['binary_week'] = pd.to_datetime(test_df['date_time']).dt.weekday

In [ ]:
print('월요일 : ' + str(train_df.query("binary_week == 0")['number_of_rentals'].sum()))
print('화요일 : ' + str(train_df.query("binary_week == 1")['number_of_rentals'].sum()))
print('수요일 : ' + str(train_df.query("binary_week == 2")['number_of_rentals'].sum()))
print('목요일 : ' + str(train_df.query("binary_week == 3")['number_of_rentals'].sum()))
print('금요일 : ' + str(train_df.query("binary_week == 4")['number_of_rentals'].sum()))
print('토요일 : ' + str(train_df.query("binary_week == 5")['number_of_rentals'].sum()))
print('일요일 : ' + str(train_df.query("binary_week == 6")['number_of_rentals'].sum()))

In [3]:
x_train =  train_df.drop(['date_time', 'number_of_rentals'], axis=1)
y_train = train_df['number_of_rentals']

x_test =  test_df.drop(['date_time'], axis=1)

In [ ]:
train_df = pd.read_csv('./dataset/train.csv')

corr = train_df.corr()
plt.figure(figsize=(10, 10));
sns.heatmap(corr,
            vmax=0.8,
            linewidths=0.01,
            square=True,
            annot=True,
            cmap='YlGnBu');
plt.title('Feature Correlation');

In [ ]:
train_df.drop(['wind_direction', 'humidity'], axis=1, inplace=True)
test_df.drop(['wind_direction', 'humidity'], axis=1, inplace=True)

In [4]:
train_df.drop(['humidity'], axis=1, inplace=True)
test_df.drop(['humidity'], axis=1, inplace=True)

In [5]:
from sklearn.preprocessing import PolynomialFeatures

x_train = PolynomialFeatures(degree=3, include_bias=False).fit_transform(x_train)
x_test = PolynomialFeatures(degree=3, include_bias=False).fit_transform(x_test)

#### RandomForestClassifier

In [6]:
from sklearn.ensemble import RandomForestClassifier

params = {
    'max_depth': [2, 3],
    'min_samples_split': [2, 3]
}

rf_clf = RandomForestClassifier()
#rf_clf = GridSearchCV(rf_clf, param_grid=params, cv=2, refit=True)

rf_clf.fit(x_train, y_train)
y_pred = rf_clf.predict(x_train)

nmae = np.mean(abs(y_pred - y_train) / y_train) # nmae 계산

print(f'모델 NMAE: {nmae}')

모델 NMAE: 0.0


#### XGBClassifier

In [ ]:
from xgboost import XGBRegressor

xgb_clf = XGBRegressor(n_estimators=5)
xgb_clf = GridSearchCV(xgb_clf, param_grid=params, cv=2, refit=True)
xgb_clf.fit(x_train, y_train)

y_pred = xgb_clf.predict(x_train)

nmae = np.mean(abs(y_pred - y_train) / y_train) # nmae 계산

print(f'모델 NMAE: {nmae}')

In [ ]:
xgb_clf = XGBRegressor(n_estimators=5)
xgb_clf.fit(x_train, y_train)
y_pred = xgb_clf.predict(x_train)

nmae = np.mean(abs(y_pred - y_train) / y_train) # nmae 계산

print(f'모델 NMAE: {nmae}')

In [7]:
submission  = pd.read_csv('dataset/sample_submission.csv')
submission

,date_time,number_of_rentals
0,2021-04-01,0
1,2021-04-02,0
2,2021-04-03,0
3,2021-04-04,0
4,2021-04-05,0
...,...,...
86,2021-06-26,0
87,2021-06-27,0
88,2021-06-28,0
89,2021-06-29,0


In [8]:
submission  = pd.read_csv('dataset/sample_submission.csv')

y_pred = rf_clf.predict(x_test)
y_pred = y_pred.round()
submission['number_of_rentals'] = y_pred

In [9]:
submission

,date_time,number_of_rentals
0,2021-04-01,39036
1,2021-04-02,79583
2,2021-04-03,2833
3,2021-04-04,26484
4,2021-04-05,60925
...,...,...
86,2021-06-26,67346
87,2021-06-27,36945
88,2021-06-28,63032
89,2021-06-29,63032


In [10]:
submission.to_csv('four.csv', index= False)